<a href="https://colab.research.google.com/github/ainem-m/AHC011/blob/main/AHC011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AHC011 17M点解法

著者: [ainem](https://atcoder.jp/users/ainem)  
自由に改変、提出していただいて構いません。

---


# 最初に実行してください

In [ ]:
from typing import Generator, List, Set, Tuple, Dict # 型ヒント用のモジュール

Coordinate = Tuple[int, int] # 座標の型を指定

# 疑似input 実行だけして読み飛ばしてください
# notebookだと入力するのがだるいので、文字列リテラルを受け取れるようにしてあります
# 提出時はDEBUG = Falseにしてください

DEBUG = True

# 他のケースを試したい時はtest_caseの内容を変える
test_case = """
6 432
62ce43
a068f9
a89da9
5d93cb
276253
424ba8
"""
readline = iter(test_case.strip("\n").split("\n"))
if DEBUG: input = lambda: next(readline)

# 実装の説明

### 入力
まずは入力を受け取るところから説明。

N: パズルの大きさ($N \times N$マス、$6 \le N \le 10$)  
T: 最大操作数($2 \times N ^{3}$)  
matrix: $N字\times N行$の文字列  

$\begin{matrix}
t_{0,0}t_{0,1}...t_{0, n-2}t_{0, n-1},  \\
t_{1,0}t_{1,1}...t_{1, n-2}t_{1, n-1},  \\
...,  \\
t_{n-2,0}t_{n-2,1}...t_{n-2, n-2}t_{n-2, n-1},  \\
t_{n-1,0}t_{n-1,1}...t_{n-1, n-2}t_{n-1, n-1} 
\end{matrix}$

実際には以下のような形
```
6 432
62ce43
a068f9
a89da9
5d93cb
276253
424ba8
```
盤面はN文字N行の文字列  
16進数なので$int(i, 16)$でint型に変換できる（そうすることで、str型より扱いが楽になる）  
二次元配列の方がわかりやすいが、処理速度に違いが出てくるので  
matrix[x][y] -> matrix[10*x + y]と置くことにして入力を受け取る


N, Tの受け取りがわからない方は方は[Python3の標準入力やり方まとめ - Qiita](https://qiita.com/yasu_teco/items/e8db933ac4f647166996)などを読んでください。  
matrixのr行目c列目の値は$matrix[10*r + c]$に格納されます  
以下が実装です↓

In [ ]:
N, T = map(int, input().split())
matrix = [-1 for _ in range(N * 10)]
for r in range(N):
    matrix[10 * r : 10 * r + N] = [int(i, 16) for i in input()]

下のセルを実行して入力を受け取れたか確認しましょう

In [ ]:
print("N, T =", N, T)
print("matrix =", matrix)

N, T = 6 432
matrix = [6, 2, 12, 14, 4, 3, -1, -1, -1, -1, 10, 0, 6, 8, 15, 9, -1, -1, -1, -1, 10, 8, 9, 13, 10, 9, -1, -1, -1, -1, 5, 13, 9, 3, 12, 11, -1, -1, -1, -1, 2, 7, 6, 2, 5, 3, -1, -1, -1, -1, 4, 2, 4, 11, 10, 8, -1, -1, -1, -1]


**これで入力を受け取ることができました！**
え、matrixが読みにくい？読みにくいですよね。あとでなんとかします。

### マスの連結判定

$\def\arraystretch{1.5}
   \begin{array}{c:c:c}
     & (k>>1)\&1 &   \\ \hline
    (k>>0)\&1 & k  & (k>>2)\&1  \\
   \hdashline
     & (k>>3)\&1 &  
\end{array}$  
i=0: 左 i=1:上 i=2: 右 i=3下として  
$(k>>i)\&1 == 1$ならば繋がりがある  

早速実装しましょう。

In [ ]:
def has_root(matrix: List[int], coordinate:Coordinate, direction:int) -> bool:
    """座標のマスから指定した方向に繋がりがあるか判定

    Args:
        matrix (List[int]): 盤面
        coordinate (Coordinate): 座標
        direction (int): 方向 左から時計回り[0:L, 1:U, 2:R, 3:D]

    Returns: bool
    """
    x, y = coordinate
    return (matrix[10 * x + y] >> direction) & 1 == 1

```
 012345
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
```

先程受け取った盤面のの3行目1列目で実行してみます。  
ピースの形は┬ですので、左、上、右、下の順番に[True, False, True, True]の結果になるはずです。

In [ ]:
print(has_root(matrix, (3, 1), 0)) # True
print(has_root(matrix, (3, 1), 1)) # False
print(has_root(matrix, (3, 1), 2)) # True
print(has_root(matrix, (3, 1), 3)) # True

True
False
True
True


この盤面、どこまでピースが連結なのかを探索したくなりますよね。そこで盤面とマスを引数として与えた時に、連結している隣接したマスを列挙してくれる関数を作ります。 

まずは隣接する４つのマスについて、現在のマスとの差分dx,dyを考えると
 * $i=0$のとき（左）$dx=0, dy=-1$
 * $i=1$のとき（上）$dx=-1, dy=0$
 * $i=2$のとき（右）$dx=0, dy=1$
 * $i=3$のとき（下）$dx=1, dy=0$

これは差分のリストDX, DYを作り
```python
DX = [0, -1, 0, 1]
DY = [-1, 0, 1, 0]
for i, (dx, dy) in enumerate(zip(DX, DY)):
  #i: 方向
  pass 
```
とfor文を回すことで４つの隣接マスについて探索できます。


隣り合ったマスそれぞれで  
 * 自分のマスから隣のマスの方向に道があるか？
 * 隣のマスから自分のマスの方向に道があるか？？

の２つを判定し、上記条件を満たせば**方向と座標**を返すことにします。


**$(i + 2)\%4$で方向$i$の反対側**を求めることができるので、

例えば$matrix[10 * x + y] = 5, matrix[10 * x + y-1] = 4$のとき  
マス(x, y)が方向$i=0$(左)のマスと繋がる条件は

 - $(5>>0)\&1 == 1$(現在のマスから左へ道が伸びている)  
 - $(4>>(0+2) \% 4) \& 1 == 1$ (左のマスから現在のマスへ道が伸びている)

と表すことができます。この例ではこの２つのマスは連結です。

また、隣のマスを探索する際に、範囲外参照をしてしまうことがありますので、その対策もします。  
対策法として外壁（番兵）を置く方法と、範囲内に収まるか判定する関数を作る方法があります。。
私は関数を作るほうが楽で好きなので、そちらを今回は採用します。

In [ ]:
def is_in(coordinate: Coordinate) -> bool:
    """座標が盤面内に収まるか判定

    Args: coordinate (Coordinate): 座標

    Returns: bool
    """
    x, y = coordinate
    return 0 <= x < N and 0 <= y < N
# DX[i]: 方向iのxの差分
DX = [0, -1, 0, 1]
DY = [-1, 0, 1, 0]

def get_adjacent(matrix: List[int], coordinate: Coordinate) -> Generator[Tuple[int, Coordinate], None, None]:
    """指定した座標から連結しているマスを列挙

    Args:
        matrix (List[int]): 盤面
        coordinate (Coordinate): 座標

    Yields:
        Generator[Tuple[int, Coordinate], None, None]: 座標からみた連結している方向と連結しているマスの座標のタプル
    """
    x, y = coordinate
    for i, (dx, dy) in enumerate(zip(DX, DY)):
        if has_root(matrix, (x, y), i):  # そのマスから方向iへ線が伸びている
            nx = x + dx
            ny = y + dy
            if not is_in((nx, ny)):
                continue
            j = (i + 2) % 4
            if has_root(matrix, (nx, ny), j):  # 隣のマスから現在のマスへ線が伸びている
                yield i, (nx, ny)

```
 012345
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
```

先程受け取った盤面のの3行目1列目で実行してみます。  
ピースの形は┬で、左と右、下のマスに連結しているので３つの方向とマスが出力されれば成功です。

In [ ]:
for i, (x, y) in get_adjacent(matrix, (3, 1)):
  print(f"i={i}, coordinate={x, y}")

i=0, coordinate=(3, 0)
i=2, coordinate=(3, 2)
i=3, coordinate=(4, 1)


### 最大の木の大きさと含まれる頂点

ここまでで盤面をグラフのように考える準備が整ったので、dfsの関数を書いていきます。  
DFSって何？って方は、[けんちょんさんの記事](https://qiita.com/drken/items/4a7869c5e304883f539b)とかが参考になると思います(c++ですが…)  
与えられた盤面と座標から、訪れたマスを記録しながら連結するマスを探索していき、頂点数と頂点の集合を返します。途中、一度訪れたマスにたどり着いた場合は閉路ありとして頂点数を0としています。

[B - バウムテスト](https://atcoder.jp/contests/arc037/tasks/arc037_b)が参考になります。

In [ ]:
def dfs_norec(matrix: List[int], seen: List[bool], coordinate: Coordinate):
    """座標から盤面をDFSして座標から繋がる木の頂点数と頂点の集合を返す。閉路がある場合頂点数は0となる

    Args:
        matrix (List[int]): 盤面
        seen (List[bool]): 訪問済の頂点
        coordinate (Coordinate): 開始地点の座標

    Returns:
        Tuple[int, Set[int]]: 木の頂点数、木の頂点の集合
    """
    x, y = coordinate
    cnt = 0 # 頂点数
    flag = True # 閉路の場合False
    q = [(x, y, -1)] # 開始地点なので方向が存在しないので-1
    tree = set() # 頂点集合
    while q:
        x, y, d = q.pop()
        if seen[10 * x + y]:
            # 一度訪れたマスにたどり着いた
            flag = False # 閉路あり
            continue
        seen[10 * x + y] = True
        tree.add(10 * x + y)
        cnt += 1
        for i, (nx, ny) in get_adjacent(matrix, (x, y)):
            if (i + 2) % 4 == d:
                # 前の方向の反対には進まない＝前の頂点には戻らない
                continue
            if seen[10 * nx + ny]:
                flag = False
                continue
            q.append((nx, ny, i))
    return cnt * flag, tree # flagががFalseならcntが0になる


```
 012345
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
```

早速先程受け取った盤面のの3行目1列目で実行してみます。５つのマスに繋がる木になっているはずです。はずです。  
DFS、BFSはとにかくバグりやすいので（個人の感想）テストは大事だと思います。。（普段私がテストするかというと…）  

In [ ]:
print(dfs_norec(matrix, [False] * (N*10), (3, 1)))

(5, {32, 41, 42, 30, 31})


この関数を使って、盤面を全探索し、盤面内の最大の木の大きさと含まれる頂点を求める関数を実装します。

In [ ]:
def get_tree(matrix: List[int]) -> Tuple[int, Set[int]]:
    """盤面から最大の木を探し、頂点数と頂点集合を返す

    Args:
        matrix (List[int]): 盤面

    Returns:
        Tuple[int, Set[int]]: 頂点数、頂点集合
    """
    seen = [False] * (N * 10)
    tree_size = 0
    tree = set()
    for i in range(N):
        for j in range(N):
            if seen[10 * i + j]:
                continue
            cnt, t = dfs_norec(matrix, seen, (i, j))
            if cnt > tree_size:
                tree_size = cnt
                tree = t
    return tree_size, tree

```
 012345
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
```

この盤面で実行すると、右下の木が検出されます（さっきの木と同率１位）

In [ ]:
print(get_tree(matrix))

(5, {34, 35, 44, 45, 25})


ほぼ同じですが、盤面の評価値を求める関数も実装します。最大の木の頂点数を基準とし、他の木の大きさの和も考慮するようにしています。評価値の考え方に関しては多分ヒューリスティックコンテストのキモになる部分ではないかと思っています。色々弄った結果、以下の評価に落ち着きました。

In [ ]:
SCORE_RATIO = 0.5
def get_score(matrix: List[int], ratio: float) -> float:
    """盤面の評価値を計算する

    Args:
        matrix (List[int]): 盤面
        ratio (float): 経過時間/時間制限

    Returns:
        float: 盤面の最大の木の頂点数 + (盤面の頂点数3以上の木の頂点数の和 * (SCORE_RATIO - ratio) / 100)
        序盤は最大の木以外の木が多いほど良い評価、終盤に近づいたら一番大きな木以外に木があると低評価になる
    """
    seen = [False] * (N * 10)
    max_size = 0
    sum_size = 0
    for i in range(N):
        for j in range(N):
            if seen[10 * i + j]:
                continue
            s, _ = dfs_norec(matrix, seen, (i, j)) # 返り値の２つ目、頂点集合は使わないので"_"で受け取って使わない
            if max_size < s:
                max_size = s
            sum_size += max(s - 3, 0)
    return max_size + (sum_size * (SCORE_RATIO - ratio) / 100)

序盤はたくさん木がある方が評価が高く、時間経過が進むほど低評価になります。

In [ ]:
print(get_score(matrix, 0))
print(get_score(matrix, 1))

5.02
4.98


### ルート生成
盤面からルート候補を生成する

まずはルートの始点となる空きマスの位置を調べる関数を実装

In [ ]:
def get_empty(matrix: List[int]) -> Coordinate:
    """盤面から空きマスの座標を探す

    Args: matrix (List[int]): 盤面

    Returns: Coordinate: 空きマスの座標
    """
    tmp = matrix.index(0)
    return tmp // 10, tmp % 10

```
 012345
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
```
空きマスを＃で表示しているので、、(1, 1)と表示されれば成功です

In [ ]:
print(get_empty(matrix))

(1, 1)


In [ ]:
import random

random.seed(19880803) # 実行ごとに結果が変わらないようにseed値を固定

TEST_TIME = 10 # 一度に生成するルートの数
ROOT_LENGTH = 5 # ルートの長さ

def make_roots(matrix: List[int]) -> Generator[str, None, None]:
    """盤面から長さN*ROOT_LENGTHのルートをTEST_TIME個生成

    Args:
        matrix (List[int]): 盤面

    Yields:
        Generator[str, None, None]: ルート
    """
    for _ in range(TEST_TIME):
        tmp = ""
        x, y = get_empty(matrix)  # 現在位置(emptyの位置は更新しない)
        loop_cnt = 0
        while len(tmp) < ROOT_LENGTH:
            if loop_cnt == 100:
                # 行き止まりになった時の無限ループ回避
                break
            loop_cnt += 1
            i = random.randint(0, 3)
            if tmp and tmp[-1] == "LURD"[(i+2)%4]:
                # 往復移動はしない
                continue
            dx, dy = DX[i], DY[i]
            if is_in((x + dx, y + dy)):
                x += dx
                y += dy
                tmp += "LURD"[i]
        yield tmp

これで

In [ ]:
for root in make_roots(matrix):
  print(root)

ULDRR
LDDDD
RRRDL
LDDRU
LURDD
RDRRR
DRUUL
DLDRU
LURRD
DRRDL


盤面からはみ出さず、往復移動をしないルートをTEST_TIME個生成することができました！

### 盤面クラスの実装
やっとここまでたどり着いた

クラスについて詳しい説明は…どなたかいい記事を教えていただけませんでしょうか…理解の難しい概念です…  
内部に盤面に関する情報を複数持ち、盤面を更新する関数や盤面の情報を整形して出力してくれるようなすてきなオブジェクトを作ります。  
詳しくは実装のあとに…

In [ ]:
class Board:
    """
    N: int 盤面の大きさ
    matrix: List[int] 盤面 高速化のため一次元化(matrix[x][y] -> matrix[10*x + y])
    empty: Tuple[int, int] 空きマスの座標
    tree_size: int 最大の木の大きさ
    tree: set[int] 最大の木の頂点の集合
    """
    
    def __init__(self, N:int, matrix:List[int]) -> None:
        self.N = N
        self.matrix = matrix
        self.empty = get_empty(self.matrix)
        self.tree_size, self.tree = get_tree(self.matrix)
    
    debug_line = [
      "#",  # 空きマス
      chr(int("2574", 16)),  # 1 > ╴
      chr(int("2575", 16)),  # 2 > ╵
      chr(int("2518", 16)),  # 3 > ┘
      chr(int("2576", 16)),  # 4 > ╶
      chr(int("2500", 16)),  # 5 > ─
      chr(int("2514", 16)),  # 6 > └
      chr(int("2534", 16)),  # 7 > ┴
      chr(int("2577", 16)),  # 8 > ╷
      chr(int("2510", 16)),  # 9 > ┐
      chr(int("2502", 16)),  # a > │
      chr(int("2524", 16)),  # b > ┤
      chr(int("250c", 16)),  # c > ┌
      chr(int("252c", 16)),  # d > ┬
      chr(int("251c", 16)),  # e > ├
      chr(int("253c", 16)),  # f > ┼
    ]

    def __str__(self):
        matrix_ = []
        for i in range(self.N):
            matrix_.append(self.matrix[10 * i : 10 * i + self.N])
        matrix = "\n".join(
            [str(i) + "".join(self.debug_line[e] for e in m) for i, m in enumerate(matrix_)]
        )
        return f"N={self.N}\nmatrix = \n 0123456789\n{matrix}\nempty = {self.empty}\ntree_size = {self.tree_size}\ntree={self.tree}"

    def move_simu(self, moves: str) -> Tuple[List[int], int]:
        """ルートを受け取りルート通り移動した盤面とルートの評価値を返す

        Args:
            moves (str): ルート

        Raises:
            ValueError: 盤面外に移動するルートの場合エラーを返す

        Returns:
            Tuple[List[int], int]: 盤面、評価値
        """
        matrix = self.matrix[:] # スライスを使ってコピー、参照渡しにならないように注意
        x, y = self.empty
        min_x = max_x = x
        min_y = max_y = y
        for m in moves:
            i = "LURD".find(m)
            dx, dy = list(zip(DX, DY))[i]
            if not is_in((x + dx, y + dy)):
                raise ValueError
            matrix[10 * x + y], matrix[10 * (x + dx) + y + dy] = (
                matrix[10 * (x + dx) + y + dy],
                matrix[10 * x + y],
            )
            x, y = x + dx, y + dy
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
        return matrix, (max_x - min_x) * (max_y - min_y)

    def move(self, moves: str) -> None:
        """ルートを受け取りクラス内の盤面を更新

        Args:
            moves (str): ルート
        """
        matrix, _ = self.move_simu(moves)
        self.matrix = matrix[:]
        self.empty = get_empty(self.matrix)
        self.tree_size, self.tree = get_tree(self.matrix)


#### デバッグ用関数

[wikipedia](https://ja.wikipedia.org/wiki/罫線素片)でUNICODEを調べて数字とピースの対応表作り
```python
debug_line: List[str] = [
  "#",  # 空きマス
  chr(int("2574", 16)),  # 1 > ╴
  chr(int("2575", 16)),  # 2 > ╵
  chr(int("2518", 16)),  # 3 > ┘
  chr(int("2576", 16)),  # 4 > ╶
  chr(int("2500", 16)),  # 5 > ─
  chr(int("2514", 16)),  # 6 > └
  chr(int("2534", 16)),  # 7 > ┴
  chr(int("2577", 16)),  # 8 > ╷
  chr(int("2510", 16)),  # 9 > ┐
  chr(int("2502", 16)),  # a > │
  chr(int("2524", 16)),  # b > ┤
  chr(int("250c", 16)),  # c > ┌
  chr(int("252c", 16)),  # d > ┬
  chr(int("251c", 16)),  # e > ├
  chr(int("253c", 16)),  # f > ┼
]
```
\_\_str\_\_という、クラスをprintしたときに呼び出される関数を実装します。  
一次元の配列になっているmatrixを二次元に戻して
```python
matrix_ = []
for i in range(self.N):
    matrix_.append(self.matrix[10 * i : 10 * i + self.N])
matrix = "\n".join(
    [str(i) + "".join(
      self.debug_line[e] for e in m
      ) for i, m in enumerate(matrix_)]
)
return f"N={self.N}\nmatrix = \n 0123456789\n{matrix}\nempty = {self.empty}\ntree_size = {self.tree_size}\ntree={self.tree}"
```
ゴニョゴニョして実装です。これはどうやったらもうすこしきれいに実装できますかね。。

ナニワトモアレ、早速Boardクラスをインスタンス化して、盤面の状態を出力させてみましょう！

In [ ]:
board = Board(N, matrix)
print(board)

N=6
matrix = 
 0123456789
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
empty = (1, 1)
tree_size = 5
tree={34, 35, 44, 45, 25}


私は盤面の移動の実装に関して、次のように２つにわけて考えました  
 * 与えられたルートの通りに操作した盤面を返す関数
 * 与えられた盤面にクラスの状態を更新する関数

２つに分けることにより、たくさんのルート候補それぞれに対して盤面を更新することなく、評価だけを行い、最良のルートだけを選択肢、盤面クラスの状態を更新することができると考えました。

```python
def move_simu(self, moves):
    """ルートを受け取りルート通り移動したmatrixとルートの評価値を返す

    Args:
        moves (str): ルート

    Raises:
        ValueError: 盤面外に移動するルートの場合エラーを返す

    Returns:
        Tuple[List[int], int]: 盤面、ルートの評価値(ルートの横幅*ルートの縦幅縦幅)
    """
    matrix = self.matrix[:] # スライスを使ってコピー、参照渡しにならないように注意
    x, y = self.empty
    min_x = max_x = x
    min_y = max_y = y
    for m in moves:
        i = "LURD".find(m)
        dx, dy = list(zip(DX, DY))[i]
        if not is_in((x + dx, y + dy)):
            raise ValueError
        # 空きマスと移動先の値をスワップ
        matrix[10 * x + y], matrix[10 * (x + dx) + y + dy] = (
            matrix[10 * (x + dx) + y + dy],
            matrix[10 * x + y],
        )
        x, y = x + dx, y + dy
        min_x = min(min_x, x)
        max_x = max(max_x, x)
        min_y = min(min_y, y)
        max_y = max(max_y, y)
    return matrix, (max_x - min_x) * (max_y - min_y)
```
後者の関数は特に変なことをしていないので説明は飛ばします。

In [ ]:
print(board)

N=6
matrix = 
 0123456789
0└╵┌├╶┘
1│#└╷┼┐
2│╷┐┬│┐
3─┬┐┘┌┤
4╵┴└╵─┘
5╶╵╶┤│╷
empty = (1, 1)
tree_size = 5
tree={34, 35, 44, 45, 25}


In [ ]:
print(board.move_simu("DDDD"))
print(get_score(board.move_simu("DDDD")[0], 0.5))
board.move("DDDD")
print(board)

([6, 2, 12, 14, 4, 3, -1, -1, -1, -1, 10, 8, 6, 8, 15, 9, -1, -1, -1, -1, 10, 13, 9, 13, 10, 9, -1, -1, -1, -1, 5, 7, 9, 3, 12, 11, -1, -1, -1, -1, 2, 2, 6, 2, 5, 3, -1, -1, -1, -1, 4, 0, 4, 11, 10, 8, -1, -1, -1, -1], 0)
6.0
N=6
matrix = 
 0123456789
0└╵┌├╶┘
1│╷└╷┼┐
2│┬┐┬│┐
3─┴┐┘┌┤
4╵╵└╵─┘
5╶#╶┤│╷
empty = (5, 1)
tree_size = 6
tree={32, 42, 21, 22, 30, 31}


In [ ]:
#board.move("D") # 盤面の外に行こうとするとValueError

### メインループの実装


### 経過時間で処理を打ち切る関数
```Python
from time import time
start_time = time() # 計測開始した時間
elapsed_time = time() - start_time # 経過時間
```
これはとってもかんたんだった
https://note.nkmk.me/python-datetime-timedelta-measure-time/

In [ ]:
from time import time

start_time: float = time()
TIME_LIMIT: float = 2.5
time_now: float = 0.0
ans: str = ""

# メインループ
while time_now <= TIME_LIMIT:
    if len(ans) > T - ROOT_LENGTH * N:
        break
    ratio: float = time_now / TIME_LIMIT
    tmp: str = ""

    roots = make_roots(board.matrix)
    score = get_score(board.matrix, ratio)
    for root in roots:
        matrix_tmp, root_area = board.move_simu(root)
        sc = get_score(matrix_tmp, ratio)
        sc += (root_area / 50) * (time_now - 1) / TIME_LIMIT
        if sc > score:
            tmp = root
            score = sc

    if tmp:
        ans += tmp
        board.move(tmp)
    time_now = time() - start_time

print(ans)
print(board)

LUURURRDLURDDRRULUULURDLDRULDLRDRRDDLLLLRRRRUDLLLLLURDLRULDRRRRRUDLLLLRRRRUDLLLLURUULURURRDLLURDLULLRDRULLDRURDLLURLDRURDDLURULLDRLURDRRULLLRRRDLLLURRRDRRUDDDLLRRUUULDLLUDRULLRDLURLDRRULLLDRLURDDURULDURDLDUURDLRULDRULDRUDLURDLURDLRULDRULDRULDDRUULDRULDRDLUURDLURDLURDLURDLURDLURDLDRUULRDLURDDLURLDRUUDLURDLURDLURDLDURDLUURDLDURULDRULDDURULDRULDRLURDLRULDRULDRUDLURDLURDLURDLDUURDLURDLDUURDLURDLDUURDLRULDD
N=6
matrix = 
 0123456789
0┐└╵└┐┐
1│╶╷┌┼┘
2│#╷╵├┐
3┴┬┘│┌┤
4─└┬─┘╷
5╵╵╶╶┤│
empty = (2, 1)
tree_size = 22
tree={0, 3, 4, 5, 10, 13, 14, 15, 20, 22, 23, 24, 25, 30, 31, 32, 34, 35, 41, 42, 43, 44}


***いかがでしたでしょうか？***

kaggleのノートブック共有のように、写経していただけたら幸いです！！
パラメータをTEST_TIME = 100, ROOT_LENGTH = N*2.5 に変更して提出すると、多分17M点がとれるはずです。。